# baseline

將`關鍵字`比對換成`向量相似度`比對。

> 請將目前使用關鍵字比對的 route_by_query，改為使用向量相似度進行分類，並設一個合理的相似度門檻，根據檢索結果的分數判斷是否走 RAG 流程。  
例如用向量相似度及自訂 threshold 決定要不要分到 retriever。

> Hint：similarity_search_with_score(...)  
可參考去年的讀書會 R4：向量資料庫的基本操作

In [1]:
from typing import Annotated, TypedDict, List

from langchain_core.documents import Document
from langchain_core.messages import HumanMessage, AIMessage
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END, START
from langgraph.graph.message import add_messages

LLM_MODEL_NAME =  "Qwen/Qwen3-4B"
EMBED_MODEL_NAME = "BAAI/bge-m3"
INFERENCE_TOP_P = 0.95
INFERENCE_TEMPERATURE = 0.0001
docs_text = """
火影代數	姓名	師傅	徒弟
初代	千手柱間	無明確記載	猿飛日斬、水戶門炎、轉寢小春
二代	千手扉間	千手柱間（兄長）	猿飛日斬、志村團藏、宇智波鏡等
三代	猿飛日斬	千手柱間、千手扉間	自來也、大蛇丸、千手綱手（傳說三忍）
四代	波風湊	自來也	旗木卡卡西、宇智波帶土、野原琳
五代	千手綱手	猿飛日斬	春野櫻、志乃等（主要為春野櫻）
六代	旗木卡卡西	波風湊	漩渦鳴人、宇智波佐助、春野櫻（第七班）
七代	漩渦鳴人	自來也、旗木卡卡西	木葉丸等（主要為木葉丸）
"""

In [2]:
# Models
chat_llm = ChatOpenAI(
    openai_api_key="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",
    openai_api_base="https://openrouter.ai/api/v1",
    model_name="mistralai/mistral-small-3.1-24b-instruct:free",
    temperature=0.1
)

embeddings = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    model_kwargs={"device": "mps"},
    encode_kwargs={"normalize_embeddings": True})


# Retriever
spliter = RecursiveCharacterTextSplitter(
    chunk_size=20,
    chunk_overlap=0,
    separators=["\n\n", "\n"],
)
docs = spliter.split_text(docs_text)
docs = spliter.create_documents(docs)

vectorstore = Chroma.from_documents(docs, embeddings, collection_name="naruto")
retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs=dict(
        score_threshold=0.2) )

/Users/roysung/miniforge3/envs/testenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# 定義 LangGraph 的 State 結構
class RAGState(TypedDict):
    query: str
    docs: List[Document]
    answer: str


def retrieve_node(state: RAGState) -> RAGState:
    query = state["query"]
    docs = retriever.invoke(query)
    return {"docs": docs}


def generate_node(state: RAGState) -> RAGState:
    query, docs = state["query"], state["docs"]
    context = "\n".join([d.page_content for d in docs])
    prompt = (
        f"你是一個知識型助手，請根據以下內容回答問題：\n\n"
        f"內容：{context}\n\n"
        f"問題：{query}\n\n回答："
    )
    output = chat_llm.invoke(prompt)
    return {"answer": output.content}


def direct_generate_node(state: RAGState) -> RAGState:
    query = state["query"]
    prompt = f"請回答以下問題：{query}\n\n回答："
    output = chat_llm.invoke(prompt).content
    return {"answer": output}


# 定義 Route Node（決定走哪條路）
def route_by_query(state):
    query = state["query"]
    docs = retriever.invoke(query)

    choice = "general"
    if docs:
        choice = "naruto"
    print(f"跑到 → {choice}")
    return choice

# 建立 LangGraph 流程圖
graph_builder = StateGraph(RAGState)
graph_builder.add_node("retriever", retrieve_node)
graph_builder.add_node("generator", generate_node)
graph_builder.add_node("direct_generator", direct_generate_node)

# 設定條件分流
graph_builder.add_conditional_edges(
    source=START,
    path=route_by_query,
    path_map={
        "naruto": "retriever",
        "general": "direct_generator",
    }
)

# 接下來的正常連接
graph_builder.add_edge("retriever", "generator")
graph_builder.add_edge("generator", END)
graph_builder.add_edge("direct_generator", END)

# 編譯 Graph
graph = graph_builder.compile()

In [7]:
print("開始對話吧（輸入 q 結束）")

while True:
    user_input = input("使用者: ")
    if user_input.strip().lower() in ["q", "quit", "exit"]:
        print("掰啦！")
        break

    init_state: RAGState = {"query": user_input}

    result = graph.invoke(init_state)
    raw_output = result["answer"]

    answer_text = raw_output.split("回答：")[-1].strip()
    print("回答：", answer_text)
    print("===" * 20, "\n")

開始對話吧（輸入 q 結束）
跑到 → naruto
回答： 根據你提供的內容，並沒有明確提到第四代火影的姓名。如果你有更多的資訊或需要進一步的解釋，請提供更多的背景資料或詳細內容。在《火影忍者》的世界觀中，第四代火影是波風水門（Minato Namikaze），他也是漩渦鳴人（Naruto Uzumaki）的父親。



No relevant docs were retrieved using the relevance score threshold 0.3


跑到 → general
回答： 相對論是由阿爾伯特·愛因斯坦（Albert Einstein）發明的。愛因斯坦在1905年提出了狹義相對論（Special Theory of Relativity），並在1915年提出了廣義相對論（General Theory of Relativity）。這些理論對於現代物理學有著深遠的影響，改變了我們對時間、空間和引力的理解。

掰啦！


# advance

改成能支援多輪問答（Multi-turn RAG），並能根據前面的query判斷問題。

> 請將 RAGState 加入 history 欄位，並在生成回答時，將歷史對話與當前問題一起組成 prompt。

> Hint：
```
class MultiTurnRAGState(TypedDict):  
    history: List[str]  
    query: str  
    docs: List[Document]  
    answer: str
```



In [ ]:
# 定義 LangGraph 的 State 結構
class MultiTurnRAGState(TypedDict):  
    history: Annotated[list, add_messages]
    query: str
    docs: List[Document]
    answer: str


def retrieve_node(state: MultiTurnRAGState) -> MultiTurnRAGState:
    query = ""
    for msg in state["history"]:
        if isinstance(msg, HumanMessage):
            query += (msg.content + " ")
    query += state["query"]
    docs = retriever.invoke(query)
    return {"docs": docs}


def generate_node(state: MultiTurnRAGState) -> MultiTurnRAGState:
    query = state["query"]
    docs = state["docs"]

    context = "\n".join([d.page_content for d in docs])
    prompt = (
        f"你是一個知識型助手，請根據以下內容回答問題：\n\n"
        f"內容：{context}\n\n"
        f"問題：{query}\n\n回答："
    )
    output = chat_llm.invoke(prompt)
    return {"history": [query, output], "answer": output.content}


def direct_generate_node(state: MultiTurnRAGState) -> MultiTurnRAGState:
    query = state["query"]
    prompt = f"請回答以下問題：{query}\n\n回答："
    output = chat_llm.invoke(prompt).content
    return {"answer": output}


# 定義 Route Node（決定走哪條路）
def route_by_query(state):
    query = state["query"]
    docs = retriever.invoke(query)

    choice = "general"
    if docs:
        choice = "naruto"
    print(f"跑到 → {choice}")
    return choice

# 建立 LangGraph 流程圖
graph_builder = StateGraph(MultiTurnRAGState)
graph_builder.add_node("retriever", retrieve_node)
graph_builder.add_node("generator", generate_node)
graph_builder.add_node("direct_generator", direct_generate_node)

# 設定條件分流
graph_builder.add_conditional_edges(
    source=START,
    path=route_by_query,
    path_map={
        "naruto": "retriever",
        "general": "direct_generator",
    }
)

# 接下來的正常連接
graph_builder.add_edge("retriever", "generator")
graph_builder.add_edge("generator", END)
graph_builder.add_edge("direct_generator", END)

# 編譯 Graph
graph = graph_builder.compile()

In [4]:
global_history: List[str] = []

print("開始對話吧（輸入 q 結束）")
while True:
    user_input = input("使用者: ")
    if user_input.strip().lower() in ["q", "quit", "exit"]:
        print("掰啦！")
        break

    state = {"history": global_history, "query": user_input}
    result = graph.invoke(state)

    answer = result["answer"].split("回答：")[-1].strip()
    print("AI 助理:", answer)
    print("===" * 60, "\n")

    global_history = result["history"]

開始對話吧（輸入 q 結束）
跑到 → naruto
第四代火影是誰?
[Document(id='535d970a-e830-43bd-a7e9-d930cf3f36ce', metadata={}, page_content='\n四代\t波風湊\t自來也\t旗木卡卡西、宇智波帶土、野原琳'), Document(id='bb9aba71-14da-4d68-9c32-bb591a984f5f', metadata={}, page_content='火影代數\t姓名\t師傅\t徒弟'), Document(id='c0379fb1-4a32-4566-922c-813fc92ebf88', metadata={}, page_content='\n七代\t漩渦鳴人\t自來也、旗木卡卡西\t木葉丸等（主要為木葉丸）')]
[]
AI 助理: 第四代火影是波風湊。

跑到 → naruto
第四代火影是誰? 他的師父是誰?
[Document(id='535d970a-e830-43bd-a7e9-d930cf3f36ce', metadata={}, page_content='\n四代\t波風湊\t自來也\t旗木卡卡西、宇智波帶土、野原琳'), Document(id='bb9aba71-14da-4d68-9c32-bb591a984f5f', metadata={}, page_content='火影代數\t姓名\t師傅\t徒弟')]
[HumanMessage(content='第四代火影是誰?', additional_kwargs={}, response_metadata={}, id='3eb89739-d432-453d-ac64-0a6c3343b8e1'), AIMessage(content='第四代火影是波風湊。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 117, 'total_tokens': 128, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model

/Users/roysung/miniforge3/envs/testenv/lib/python3.10/site-packages/langchain_core/vectorstores/base.py:1082: UserWarning: Relevance scores must be between 0 and 1, got [(Document(id='bb9aba71-14da-4d68-9c32-bb591a984f5f', metadata={}, page_content='火影代數\t姓名\t師傅\t徒弟'), 0.3279693434307649), (Document(id='9295b784-c9ef-4aee-84e6-e9c75c55c5cd', metadata={}, page_content='\n六代\t旗木卡卡西\t波風湊\t漩渦鳴人、宇智波佐助、春野櫻（第七班）'), 0.03837126664186219), (Document(id='f72f8bf3-528f-4383-9d80-a206e97e0148', metadata={}, page_content='\n三代\t猿飛日斬\t千手柱間、千手扉間\t自來也、大蛇丸、千手綱手（傳說三忍）'), 0.0028069129313996743), (Document(id='a1913900-84c2-4870-9aea-202ec8d34a41', metadata={}, page_content='\n二代\t千手扉間\t千手柱間（兄長）\t猿飛日斬、志村團藏、宇智波鏡等'), -0.0022564408613050535)]
  self.vectorstore.similarity_search_with_relevance_scores(


跑到 → naruto
第四代火影是誰? 他的師父是誰? 他的徒弟有哪些人?
[Document(id='bb9aba71-14da-4d68-9c32-bb591a984f5f', metadata={}, page_content='火影代數\t姓名\t師傅\t徒弟'), Document(id='535d970a-e830-43bd-a7e9-d930cf3f36ce', metadata={}, page_content='\n四代\t波風湊\t自來也\t旗木卡卡西、宇智波帶土、野原琳')]
[HumanMessage(content='第四代火影是誰?', additional_kwargs={}, response_metadata={}, id='3eb89739-d432-453d-ac64-0a6c3343b8e1'), AIMessage(content='第四代火影是波風湊。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 117, 'total_tokens': 128, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'mistralai/mistral-small-3.1-24b-instruct:free', 'system_fingerprint': None, 'id': 'gen-1747760834-iZx1kSwMNgwSIRKyLB60', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--dd0536fe-3c7a-4de4-917b-6b902990df88-0', usage_metadata={'input_tokens': 117, 'output_tokens': 11, 'total_tokens': 128, 'input_token_details': {}, 'output_token_details': {}})

In [ ]:
global_history: List[str] = []

print("開始對話吧（輸入 q 結束）")
while True:
    user_input = input("使用者: ")
    if user_input.strip().lower() in ["q", "quit", "exit"]:
        print("掰啦！")
        break

    state = {"history": global_history, "query": user_input}
    result = graph.invoke(state)

    answer = result["answer"].split("回答：")[-1].strip()
    print("AI 助理:", answer)
    print("===" * 60, "\n")

    global_history = result["history"]

開始對話吧（輸入 q 結束）
使用者: 第四代火影是誰?
route: cosine_sim = 0.8092
跑到 → retriever
retrieve combined query: '第四代火影是誰?'
AI 助理: 第四代火影是波風湊。

使用者: 他的師父是誰?
route: cosine_sim = 0.5529
跑到 → retriever
retrieve combined query: '第四代火影是誰?\n他的師父是誰?'
AI 助理: 第四代火影的師父是自來也。

使用者: 他的徒弟有哪些人?
route: cosine_sim = 0.6542
跑到 → retriever
retrieve combined query: '第四代火影是誰?\n他的師父是誰?\n他的徒弟有哪些人?'
AI 助理: 他的徒弟有以下人：旗木卡卡西、宇智波帶土、野原琳。

使用者: 相對論是他發明的嗎?
route: cosine_sim = 0.1118
跑到 → general
AI 助理: 相對論不是第四代火影所發明的。相對論是物理學家阿爾伯特·愛因斯坦在1905年提出的，他提出了廣義相對論，後人又提出狹義相對論。相對論主要是研究加速度和重力的關係，以及加速度和時間、空間的關係。

使用者: q
掰啦！
